In [1]:
import os
import numpy as np
import pandas as pd
import datetime as dt
import warnings
import time
warnings.filterwarnings(action='ignore')

In [2]:
class DataShift:
    def __init__(self, grade, data):
        self.grade = grade
        self.data_origin = data
        self.root = f'{os.getcwd()}/data'
        self.path_lag = f'{self.root}/timelag_{self.grade}_v2.csv'
        self.timelag_origin = pd.read_csv(self.path_lag)
        
                            
    def logging(self, text):
        TXT_LOG.write(f'{text}\n')
    
    
    def make_df_for_shift(self, data_sch, lag_info):
        ''' shift를 위한 큰 dataframe 생성 '''
        df_data = pd.DataFrame()
        df_data = data_sch[lag_info.index]
        df_up = pd.DataFrame(index=range(95), columns=lag_info.index)
        df_down = pd.DataFrame(index=range(110), columns=lag_info.index)
        return pd.concat([df_up, df_data, df_down], axis=0)
    
            
    def get_schedule_data(self, sch_id):
        ''' 스케쥴 id에 해당하는 data 불러오기 '''
        self.logging(f'\n = Schedule id : {sch_id}')
        origin = self.data_origin
        data_sch = origin[origin['crack_id'].isin([sch_id])] # 개별 스케쥴 데이터
        data_sch.dropna(how='all', axis=1, inplace=True)   # 모두 nan인 컬럼 제거

        # Nan 선형 보간, 처리 방법 개선 필요
        n_nan = data_sch.isnull().sum().sum()
        if  n_nan > 0:
            self.logging(f'   - Missing valus({n_nan}) -> interpolation')
            null_info = data_sch.isnull().sum()
            null_info = null_info[null_info > 0]
            for i in null_info.index:
                self.logging(f'      . {i} : {str(null_info[i])}')
            data_sch = data_sch.interpolate()
        return data_sch
    
            
    def get_lag_info(self, data, sch_id):
        ''' 변수별 time lag 정보 파일 불러오기'''
        lag_info = self.timelag_origin.set_index('tag', inplace=False)
        drop_list = []
    
        # data에 없는 관심 변수들 제거
        for var in lag_info.index:
            if var not in data.columns:
                drop_list += [var]
                lag_info.drop(var, axis=0, inplace=True)
                
        if drop_list != []:
            self.logging(f'   - Missing tags: {drop_list}')
        return lag_info
        



    def shift(self, data_sch, lag_info, sch_id):
        """ 변수별 시간 차 만큼 shift """
        data_sch = self.make_df_for_shift(data_sch, lag_info)
        for var in lag_info.index:
            if var not in data_sch.columns:
                self.logging(f'변수 {var}가 해당 스케쥴 파일에 없습니다.')
                continue
            data_sch[var] = data_sch[var].shift(periods=lag_info.loc[var][0]*-1)
            
        # NaN행 삭제, 하지막 컬럼에 스케중 id 작성(확인용)
        data_sch.dropna(axis=0, how='any', inplace=True)
        data_sch['crack_id'] = sch_id
        
        # shape, NaN  정보 저장(확인 용)
        shape_ = data_sch.shape
        self.logging(f'   - {shape_[1]} columns, {shape_[0]} samples')
        self.logging(f'   - NaN: {data_sch.isnull().sum().sum()}')
        return data_sch
    

    
    def run(self, sch_id):
        data_sch = self.get_schedule_data(sch_id)         # 스케쥴 별 tag 데이터
        lag_info = self.get_lag_info(data_sch, sch_id)    # time gap 정보 데이터
        data_sch = self.shift(data_sch, lag_info, sch_id) # 변수별 time shift
        
                
        # 소수점, 데이터 형식 처리
        for var in data_sch.columns:
            if var in ['TIME']:
                continue
            if var == 'crack_id':
                data_sch[var] = data_sch[var].astype(int)
            else:
                data_sch[var] = data_sch[var].round(4)
        
        return data_sch


In [3]:
class FailureCheck:
    def __init__(self):
        self.SMOOTH_WINDOW_SIZE = 3
        self.TEMP_THRESHOLD = -0.2
        self.BORATE_THRESHOLD = 0.32
        self.PADDING_SIZE = 3

    def get_failure_col(self, df):
        # 온도, 조촉매 차이 정의
        target_col = df.LL1TRC2205
        borate_col = df.LL1FC2302A1

        diff_temp_col = target_col - target_col.shift(1)
        diff_borate_col = borate_col - borate_col.shift(1)

        # smoothing
        diff_temp_col = diff_temp_col.rolling(self.SMOOTH_WINDOW_SIZE).mean()
        diff_borate_col = diff_borate_col.rolling(self.SMOOTH_WINDOW_SIZE).mean()

        # 온도 하락 구간 정의
        failure_col = (diff_temp_col < self.TEMP_THRESHOLD) & (
            diff_borate_col > self.BORATE_THRESHOLD
        )

        # failure 직전 구간도 failure로 정의
        failure_col = (failure_col.shift(-self.PADDING_SIZE + 1)
                       .rolling(self.PADDING_SIZE, min_periods=1).max()
                      )

        return failure_col

    def run(self, data):
        failure_col = self.get_failure_col(data)
        data["failure"] = failure_col
        
        data.dropna(how='any', axis=0, inplace=True)
        
        return data

In [4]:
class PreProcessing:
    
    def __init__(self, grade):
        self.grade = grade
        self.root = f'{os.getcwd()}/data'
        self.data_origin = pd.read_csv(f'{self.root}/data_{grade}.csv') # grade 데이터
        
        
    def concat_features(self):
        df = self.data_origin
        # Octene Feed
        df["LL1FRC2101"] = df["LL1FRC2101"] + df["LL1FRC2117"] + df["LL1FRC2105N"]
        # 601-E Fresh 옥텐 공급량
        df["LL1FRC6103"] = df["LL1FRC6103"] + df["LL1FRC6101"]
        # 601-E→602-E 유량
        df["LL1FRC6110A"] = df["LL1FRC6110A"] + df["LL1FRC6110"]
        # Solvent Ratio 합치기 (LL1FRC2107/LL1FRC2121)
        df["LL1FRC2107"] = df["LL1FRC2107"] + df["LL1FRC2121"]

        # 불필요한 칼럼 삭제
        df.drop(
            ["LL1FRC2117", "LL1FRC2105N", "LL1FRC6101", "LL1FRC6110", "LL1FRC2121"],
            axis=1,
            inplace=True,
        )

        return df


    def filter_features(self):
        df = self.concat_features()
        # nan 값이 많은 경우 삭제
        df.drop(["LL1LR6102", "LL1LIC6204"], axis=1, inplace=True)
#         valid_feature_set = ["LL1TRC2205"] + get_valid_features(list(df.columns))
#         df = df[valid_feature_set]

        return df

    
    def cut_outlier(self, data):
        df = pd.concat([data.iloc[:75000], data.iloc[138000:]], axis=0, ignore_index=True)
        return df
    
    def check_continuity(self, data):
        time_df = data['TIME']
        d_ =[]
        crack_id = 0

        for i, t in enumerate(time_df):
            if i >= time_df.shape[0] - 1:
                break
            time_f = time_df[i+1]
            time_now = time_df[i]
            time_f_str = time.strptime(time_f,'%Y-%m-%d %H:%M:%S')
            time_now_str = time.strptime(time_now,'%Y-%m-%d %H:%M:%S')
#             time_f_str = time.strptime(time_f,'%Y-%m-%d %H:%M')
#             time_now_str = time.strptime(time_now,'%Y-%m-%d %H:%M')

            time_f_mk = time.mktime(time_f_str)
            time_now_mk = time.mktime(time_now_str)

            err = 0 if time_f_mk - time_now_mk == 60 else 1
            crack_id += err
            d_ += [crack_id]

        d_ = pd.DataFrame(d_, columns=['crack_id'])

        data = pd.concat([data, d_], axis=1)
        data = data.iloc[:-1]

        return data

    
    def run(self):
        data = self.filter_features()
        data = self.cut_outlier(data)
        data = self.check_continuity(data)

        return data

In [5]:
# Global
global TXT_LOG    # 처리 결과 로그 파일
global PATH_SAVE
global TIME_ST

TIME_ST = dt.datetime.now()
PATH_SAVE = f"{os.getcwd()}/data_proc/{TIME_ST.strftime('%Y-%m-%d')}"

# 저장 폴더 생성
if not os.path.exists(PATH_SAVE):
    os.makedirs(PATH_SAVE)

# Params
grade = 670       # 분석하는 grade
pred_point = 10   # 예측 시점(+/- 분)
window_size = 60  # 데이터 처리 구간(분)
user_drop = []    # 사용자 임의로 제거하려는 tag 리스트
total_shift = pd.DataFrame() # 처리한 데이터 담을 df
total_shift_mavg = pd.DataFrame() # 이동편균 처리한 데이터 담을 df

TXT_LOG = open(f"{PATH_SAVE}/log_{grade}_{TIME_ST.strftime('%Y-%m-%d-%H-%M')}.txt", 'w')
save_name = f"shift_{grade}_{TIME_ST.strftime('%Y-%m-%d-%H-%M')}.csv"

# class 선언
PreProcessing = PreProcessing(grade) # 전처리 class(변수 삭제, 합성 등)
data = PreProcessing.run()           # 전처리 데이터
FailureCheck = FailureCheck()                # 온도 하락 라벨링 class
DataShift = DataShift(grade, data)   # 시간차 보정 class



In [6]:
def moving_avg(data_sch):
    shift_mavg = pd.DataFrame()
    for var in data_sch.columns:
        if var in ['TIME']:
            continue
        shift_mavg[var] = data_sch[var].rolling(window=5).mean()
        
        shift_mavg.dropna(how='any', axis=0, inplace=True)

        if var == 'crack_id':
            shift_mavg[var] = shift_mavg[var].astype(int)
        else:
            shift_mavg[var] = shift_mavg[var].round(4)
    return shift_mavg

def 

In [7]:
# 스케줄 별 데이터 처리
for sch_id in data['crack_id'].unique():
    data_ = DataShift.run(sch_id)            # 시간차 보정
    data_ = FailureCheck.run(data_)          # 온도하락 라벨링
    
    
    data_mavg = moving_avg(data_)
    
#     print(data_mavg)
    
    save_name_ = f"shift_{grade}_{TIME_ST.strftime('%Y-%m-%d-%H-%M')}_{sch_id}.csv"
    data_.to_csv(f'{PATH_SAVE}/{save_name_}', index=False)
    data_mavg.to_csv(f'{PATH_SAVE}/{save_name_.replace(".csv", "_m.csv", 1)}', index=False)

    total_shift = total_shift.append(data_)  # 처리한 데이터 저장
    total_shift_mavg = total_shift_mavg.append(data_mavg)

# 처리한 데이터, 로그 저장
total_shift.to_csv(f'{PATH_SAVE}/{save_name}', index=False)
total_shift_mavg.to_csv(f'{PATH_SAVE}/{save_name.replace(".csv", "_m.csv", 1)}', index=False)
TXT_LOG.close()

     LL1FIC3402  LL1FR2114  LL1FC2302A1  LL1FC2302B1  LL1FC2302C1  LL1SI2201  \
114    116.7409  1119.7846      69.3195      32.3484      19.8496   104.8932   
115    116.9127  1110.0940      69.2913      32.3368      19.9193   104.8890   
116    117.0272  1104.4946      69.2415      32.3522      19.7836   104.8904   
117    116.6676  1097.7622      69.2474      32.5979      19.7698   104.8891   
118    116.8197  1100.3188      69.1167      32.5918      19.7322   104.8835   
..          ...        ...          ...          ...          ...        ...   
620    118.4525  1125.7634      57.2835      28.8144      19.7674   104.8905   
621    118.0363  1125.6266      57.5079      28.8537      19.7463   104.8891   
622    118.4147  1121.0854      57.4456      28.9091      19.8298   104.8863   
623    119.0941  1123.4540      57.3160      28.7772      19.9809   104.8863   
624    118.2073  1119.7152      57.0586      28.7328      20.0015   104.8800   

     LL1JR2201  LL1TRC2203  LL1FIC6301 